<a href="https://colab.research.google.com/github/Mina-ALLA/Machine_Translation_Project/blob/master/Machine_translation_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Building machine translation (from English to French) using seq2seq model**


**Introduction**

In this notebook, we will build a deep neural network that functions as part of an end-to-end machine translation pipeline. this pipeline will accept English text as input and return the French translation.

**Preprocess** : we will convert text to sequence of integers.

**Models** : Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, we will engage in our own investigations, to design the final model!

**Prediction** : Runing the model on English text.

In [0]:
import collections

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout, LSTM
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


We should always verify access to the GPU to speed up the learning

**DataSet**

**Load Data**

The data is located in small_vocab_en and small_vocab_fr. The small_vocab_en file contains English sentences with their French translations in the small_vocab_fr file. we are going to load the English and French data from these files in my github from running the cell below.

In [0]:
import urllib3

link1 = 'https://raw.githubusercontent.com/Mina-ALLA/Machine_Translation_Project/master/small_vocab_en.txt'
link2 = 'https://raw.githubusercontent.com/Mina-ALLA/Machine_Translation_Project/master/small_vocab_fr.txt'
http = urllib3.PoolManager()
response_eng = http.request('GET', link1)
response_fr = http.request('GET', link2)
data_eng = response_eng.data.decode('utf-8')
data_fr = response_fr.data.decode('utf-8')
english_sentences = data_eng.split('\n')
french_sentences = data_fr.split('\n')

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [0]:
a = len(english_sentences)
b = len(french_sentences)
print(" size of english sentences and french sentences")
a,b

 size of english sentences and french sentences


(137861, 137861)

**Splitting data to training and test datasets**

In [0]:
train_eng_data, test_eng_data, train_fr_data, test_fr_data = train_test_split(english_sentences, french_sentences, test_size=0.2)
a1 = len(train_eng_data)
b1 = len(train_fr_data)
print("shape of training set : ("+str(a1)+','+str(b1)+')')

a2 = len(test_eng_data)
b2 = len(test_fr_data)
print("shape of validation set : ("+str(a2)+','+str(b2)+')')

shape of training set : (110288,110288)
shape of test set : (27573,27573)


**Exploring the files **

Each line in small_vocab_en contains an English sentence with the respective translation in each line of small_vocab_fr. we are going to view the first two lines from each file.

In [0]:
for sample_i in range(5):
    print('English sample {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('French sample {}:  {}\n'.format(sample_i + 1, french_sentences[sample_i]))

English sample 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
French sample 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .

English sample 2:  the united states is usually chilly during july , and it is usually freezing in november .
French sample 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .

English sample 3:  california is usually quiet during march , and it is usually hot in june .
French sample 3:  california est généralement calme en mars , et il est généralement chaud en juin .

English sample 4:  the united states is sometimes mild during june , and it is cold in september .
French sample 4:  les états-unis est parfois légère en juin , et il fait froid en septembre .

English sample 5:  your least liked fruit is the grape , but my least liked is the apple .
French sample 5:  votre moins aimé fruit est le raisin , mais mon moins aimé est la pomme .



From looking at the sentences, we can see they have been preprocessed already but they needs more preprocessing.

Let's explore the vocabulary of the texts and see the most frequent words.

In [0]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


**Preprocess**

For this project, we won't use text data as input to our model. Instead, we'll convert the text into sequences of integers using the following preprocess methods:

Tokenize the words into ids (one-hot encodings) #unordered discrete data
Add padding to make all the sequences the same length.

**Tokenize (IMPLEMENTATION)**

For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings. Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number. These are called character and word ids, respectively. Character ids are used for character level models that generate text predictions for each character. A word level model uses word ids that generate text predictions for each word. Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's Tokenizer function. Use this function to tokenize english_sentences and french_sentences in the cell below.

Running the cell will run tokenize on sample data and show output for debugging.

In [0]:
def tokenize(x):

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer

 # Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))   

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


**Padding (IMPLEMENTATION)**

When batching the sequence of word ids together, each sequence needs to be the same length. Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

So we make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the end of each sequence using Keras's pad_sequences function.

In [0]:

def pad(x, length=None):

  return pad_sequences(x, maxlen=length, padding='post')

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))  

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


**Preprocess Pipeline on the whole datasets**

In [0]:
def preprocess(x, y):

    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer = preprocess(english_sentences, french_sentences)

max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English train sentence length:", max_english_sequence_length)
print("Max French train sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)


Data Preprocessed
Max English train sentence length: 15
Max French train sentence length: 21
English vocabulary size: 200
French vocabulary size: 345


**Ids Back to Text**

The neural network will be translating the input to words ids, which isn't the final form we want. We want the French translation. The function logits_to_text will bridge the gab between the logits from the neural network to the French translation. We'll be using this function to better understand the output of the neural network.

In [0]:
def logits_to_text(logits, tokenizer):
  
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


**Models**

In this section, we will experiment with various neural network architectures. we will begin by training four relatively simple architectures.

**Model 1 :** is a simple RNN

**Model 2 :** is a RNN with Embedding

**Model 3 :** is a Bidirectional RNN

**Model 4 :** is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, we will construct a deeper architecture that is designed to outperform all four models.

**Model 1: Simple RNN**

In [0]:

def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    
    # Hyperparameters
    learning_rate = 0.005
    
    # TODO: Build the layers
    model = Sequential()
    model.add(GRU(256, input_shape=input_shape[1:], return_sequences=True))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax'))) 

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

print(simple_rnn_model.summary())

simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))    


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 21, 256)           198144    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 21, 1024)          263168    
_________________________________________________________________
dropout_1 (Dropout)          (None, 21, 1024)          0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 21, 344)           352600    
Total params: 813,912
Trainable params: 813,912
Non-trainable params: 0
_________________________________________________________________
None
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 12s 105us/step - loss: 1.9689 - accuracy: 0.5364 - val_loss: nan - val_accuracy: 0.6329
Epoch 2/10
110288/110288 [================

In [0]:
# Print prediction(s)
print("Prediction:")
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

print("\nCorrect Translation:")
print(french_sentences[:1])

print("\nOriginal text:")
print(english_sentences[:1])

Prediction:
new jersey est parfois chaud en mois de il est il est en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]

Original text:
['new jersey is sometimes quiet during autumn , and it is snowy in april .']


**Model 2: Using word Embedding**

We've turned the words into ids, but there's a better representation of a word. This is called word embeddings. An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, we'll create a RNN model using embedding.

In [0]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    
    # Hyperparameters
    learning_rate = 0.005
    
    # TODO: Build the layers
    model = Sequential()
    model.add(Embedding(english_vocab_size, 256, input_length=input_shape[1], input_shape=input_shape[1:]))
    model.add(GRU(256, return_sequences=True))    
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax'))) 

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# TODO: Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

embed_rnn_model.summary()

embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))    

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 21, 256)           51200     
_________________________________________________________________
gru_2 (GRU)                  (None, 21, 256)           393984    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 21, 1024)          263168    
_________________________________________________________________
dropout_2 (Dropout)          (None, 21, 1024)          0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 21, 345)           353625    
Total params: 1,061,977
Trainable params: 1,061,977
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 11s 98us/step - loss: 1.4097 - accuracy: 0.6740 - val_loss: 0.5197 - val_accuracy: 0.8334
Epoch 2/10
110288/110288 [==============================] - 10s 91us/step - loss: 0.4318 - accuracy: 0.8584 - val_loss: 0.3232 - val_accuracy: 0.8890
Epoch 3/10
110288/110288 [==============================] - 10s 92us/step - loss: 0.3081 - accuracy: 0.8967 - val_loss: 0.2535 - val_accuracy: 0.9139
Epoch 4/10
110288/110288 [==============================] - 10s 91us/step - loss: 0.2541 - accuracy: 0.9141 - val_loss: 0.2245 - val_accuracy: 0.9228
Epoch 5/10
110288/110288 [==============================] - 10s 92us/step - loss: 0.2260 - accuracy: 0.9226 - val_loss: 0.2089 - val_accuracy: 0.9273
Epoch 6/10
110288/110288 [==============================] - 10s 92us/step - loss: 0.2110 - accuracy: 0.9271 - val_loss: 0.1980 - val_accuracy: 0.9314
Epoch 7/10
110288/110288 [=======================

In [0]:
# Print prediction(s)
print("Prediction:")
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

print("\nCorrect Translation:")
print(french_sentences[:1])

print("\nOriginal text:")
print(english_sentences[:1])

Prediction:
new jersey est parfois calme en l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]

Original text:
['new jersey is sometimes quiet during autumn , and it is snowy in april .']


**Model 3: Bidirectional RNNs**

One restriction of a RNN is that it can't see the future input, only the past. This is where bidirectional recurrent neural networks come in. They are able to see the future data.

In [0]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):

    # Hyperparameters
    learning_rate = 0.003
    
    # TODO: Build the layers
    model = Sequential()
    model.add(Bidirectional(GRU(128, return_sequences=True), input_shape=input_shape[1:]))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax'))) 

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# TODO: Train and Print prediction(s)
embed_rnn_model = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

embed_rnn_model.summary()

embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))    

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 21, 256)           51200     
_________________________________________________________________
gru_3 (GRU)                  (None, 21, 256)           393984    
_________________________________________________________________
time_distributed_5 (TimeDist (None, 21, 1024)          263168    
_________________________________________________________________
dropout_3 (Dropout)          (None, 21, 1024)          0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 21, 345)           353625    
Total params: 1,061,977
Trainable params: 1,061,977
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 11s 99us/step - loss: 1.4870 - accuracy: 0.6602 - val_loss: 0.5083 - val_accuracy: 0.8379
Epoch 2/10
110288/110288 [==============================] - 10s 91us/step - loss: 0.4376 - accuracy: 0.8569 - val_loss: 0.3206 - val_accuracy: 0.8932
Epoch 3/10
110288/110288 [==============================] - 10s 91us/step - loss: 0.3150 - accuracy: 0.8948 - val_loss: 0.2613 - val_accuracy: 0.9112
Epoch 4/10
110288/110288 [==============================] - 10s 92us/step - loss: 0.2584 - accuracy: 0.9125 - val_loss: 0.2262 - val_accuracy: 0.9223
Epoch 5/10
110288/110288 [==============================] - 10s 93us/step - loss: 0.2286 - accuracy: 0.9217 - val_loss: 0.2065 - val_accuracy: 0.9282
Epoch 6/10
110288/110288 [==============================] - 10s 93us/step - loss: 0.2138 - accuracy: 0.9262 - val_loss: 0.2006 - val_accuracy: 0.9301
Epoch 7/10
110288/110288 [=======================

In [0]:
print("Prediction:")
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

print("\nCorrect Translation:")
print(french_sentences[:1])

print("\nOriginal text:")
print(english_sentences[:1])

Prediction:
new jersey est parfois calme en l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]

Original text:
['new jersey is sometimes quiet during autumn , and it is snowy in april .']


**Model 4: Encoder-Decoder**

This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence. The decoder takes this matrix as input and predicts the translation as output.

In [0]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
  
    # Hyperparameters
    learning_rate = 0.001
    
    # Build the layers    
    model = Sequential()
    # Encoder
    model.add(GRU(256, input_shape=input_shape[1:], go_backwards=True))
    model.add(RepeatVector(output_sequence_length))
    # Decoder
    model.add(GRU(256, return_sequences=True))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

# Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train and Print prediction(s)
encdec_rnn_model = encdec_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

encdec_rnn_model.summary()

encdec_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)    



Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_4 (GRU)                  (None, 256)               198144    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 21, 256)           0         
_________________________________________________________________
gru_5 (GRU)                  (None, 21, 256)           393984    
_________________________________________________________________
time_distributed_7 (TimeDist (None, 21, 1024)          263168    
_________________________________________________________________
dropout_4 (Dropout)          (None, 21, 1024)          0         
_________________________________________________________________
time_distributed_8 (TimeDist (None, 21, 345)           353625    
Total params: 1,208,921
Trainable params: 1,208,921
Non-trainable params: 0
____________________________________________

In [0]:
# Print prediction(s)
print("Prediction:")
print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

print("\nCorrect Translation:")
print(french_sentences[:1])

print("\nOriginal text:")
print(english_sentences[:1])

Prediction:
new jersey est parfois agréable en l' et il est est en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]

Original text:
['new jersey is sometimes quiet during autumn , and it is snowy in april .']


**Model 5: RNN using all above**

In [0]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):

    # Hyperparameters
    learning_rate = 0.003
    
    # Build the layers    
    model = Sequential()
    # Embedding
    model.add(Embedding(english_vocab_size, 128, input_length=input_shape[1],
                         input_shape=input_shape[1:]))
    # Encoder
    model.add(Bidirectional(GRU(128)))
    model.add(RepeatVector(output_sequence_length))
    # Decoder
    model.add(Bidirectional(GRU(128, return_sequences=True)))
    model.add(TimeDistributed(Dense(512, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

**Final Prediction**

In [0]:
def final_predictions(x, y, x_tk, y_tk):

    # TODO: Train neural network using model_final
    model = model_final(x.shape,y.shape[1],
                        len(x_tk.word_index)+1,
                        len(y_tk.word_index)+1)
    model.summary()
    model.fit(x, y, batch_size=1024, epochs=25, validation_split=0.2)

    
    ## Test sample
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print('he saw a old yellow truck')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    #print('Il a vu un vieux camion jaune')
    #print('Sample 2:')
    #print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    #print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))

final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)    

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 15, 128)           25600     
_________________________________________________________________
bidirectional_9 (Bidirection (None, 256)               197376    
_________________________________________________________________
repeat_vector_5 (RepeatVecto (None, 21, 256)           0         
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 21, 256)           295680    
_________________________________________________________________
time_distributed_9 (TimeDist (None, 21, 512)           131584    
_________________________________________________________________
dropout_5 (Dropout)          (None, 21, 512)           0         
_________________________________________________________________
time_distributed_10 (TimeDis (None, 21, 345)          

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 110288 samples, validate on 27573 samples
Epoch 1/25
110288/110288 [==============================] - 38s 342us/step - loss: 2.4466 - accuracy: 0.4811 - val_loss: 1.5981 - val_accuracy: 0.5917
Epoch 2/25
110288/110288 [==============================] - 36s 329us/step - loss: 1.4375 - accuracy: 0.6187 - val_loss: 1.1545 - val_accuracy: 0.6810
Epoch 3/25
110288/110288 [==============================] - 36s 328us/step - loss: 1.1408 - accuracy: 0.6782 - val_loss: 0.9739 - val_accuracy: 0.7169
Epoch 4/25
110288/110288 [==============================] - 36s 327us/step - loss: 1.0277 - accuracy: 0.7017 - val_loss: 0.8438 - val_accuracy: 0.7450
Epoch 5/25
110288/110288 [==============================] - 36s 322us/step - loss: 0.8660 - accuracy: 0.7380 - val_loss: 0.7323 - val_accuracy: 0.7747
Epoch 6/25
110288/110288 [==============================] - 36s 322us/step - loss: 0.7909 - accuracy: 0.7556 - val_loss: 0.6315 - val_accuracy: 0.8024
Epoch 7/25
110288/110288 [=================